In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/HCI

In [ ]:
ls

Dataset.txt  Terrorism_Related_Words.xlsx


In [ ]:
# Reading the Dataset
file_path = 'Dataset.txt'

with open(file_path, 'r') as file:
    file_content = file.read()

In [ ]:
import pandas as pd
import re

# Splitting the content into individual reports
reports = file_content.split("REPORT\n")[1:]  # [1:] to skip the first empty split

# Function to extract data from a report based on the given keys
def extract_report_data(report):
    keys = ['ID', 'REPORTDATE', 'REFERENCEID', 'REPORTSOURCE', 'REPORTDESCRIPTION', 'PERSONS', 'DATES', 'PLACES', 'ORGANIZATIONS']
    report_data = {}

    for key in keys:
        # Using regular expression to find the corresponding value for each key
        pattern = f"{key}: (.*?)\n"
        match = re.search(pattern, report, re.S)
        report_data[key] = match.group(1).strip() if match else None

    return report_data

# Extracting data for each report
extracted_data = [extract_report_data(report) for report in reports]

# Creating a DataFrame from the extracted data
df_reports = pd.DataFrame(extracted_data)



In [ ]:
# Displaying the first few rows of the DataFrame
df_reports.head(4)

In [ ]:
Dataset_report_descriptions = df_reports[['REPORTDESCRIPTION']]

In [ ]:
Dataset_report_descriptions.head(4)

In [ ]:
# Reading the Dataset
file_path = 'Terrorism_Related_Words.xlsx'

Terrorism_Related_Words = pd.read_excel(file_path)

In [ ]:
Terrorism_Related_Words

In [ ]:
# KeyBERT:

In [ ]:
pip install keybert

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()

kw_model_multi = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
all_keywords = []

for text in Dataset_report_descriptions["REPORTDESCRIPTION"]:
    temp = kw_model.extract_keywords(text, top_n=20)
    all_keywords.append(temp)

In [ ]:
Dataset_report_descriptions["REPORTDESCRIPTION"][0]

In [ ]:
all_keywords[0]

In [ ]:
len(all_keywords)

111

In [ ]:
len(all_keywords[0])

20

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

# Load pre-trained sentence transformer model
s_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
#embedding for Terrorism_Related_Words:

In [ ]:
threat_embeddings = s_model.encode(list(Terrorism_Related_Words['Word']))

In [ ]:
len(threat_embeddings)

476

In [ ]:
#embedding for Dataset key words:

In [ ]:
Terrorism_Related_Words

In [ ]:
all_keywords[0]

In [ ]:
from sentence_transformers import util

In [ ]:
# countingh the number of people:
def count_names(names_string):
    if names_string and names_string.strip():
        names_list = names_string.split(';')
        return len(names_list)
    else:
        return 0

In [ ]:
# Find the most similar words from threat_levels for each word in data_keys

final_threat_score = []
final_threat_score
dic = {}

for j, keys in enumerate(all_keywords):
    dic_temp = {}
    keys_words = [t[0] for t in keys]
    embedded_keys = s_model.encode(keys_words)
    sum = 0
    temp_level = []
    each_word_score = []
    for i, word in enumerate(keys_words):
        word_score = []
        # Compute cosine similarity between word embedding and threat embeddings
        similarities = util.cos_sim(embedded_keys[i], threat_embeddings)
        # Find the index of the highest similarity score
        most_similar_idx = similarities[0].argmax().item()
        # Get the similarity score
        similarity_score = similarities[0][most_similar_idx].item()
        # Get the most similar word and its threat level
        most_similar_word = Terrorism_Related_Words['Word'][most_similar_idx]
        threat_level =  Terrorism_Related_Words['Threat Level'][most_similar_idx]
        # Print the most similar word, its threat level, and the similarity score

        if similarity_score < 0.6:
            word_score = round(threat_level * similarity_score)
            sum = sum + word_score
        else:
            word_score = round(threat_level * threat_level)
            sum = sum + word_score

        each_word_score.append(word_score)

    final_threat_score.append(sum)

    dic_temp['ID'] = j
    dic_temp['Report'] = df_reports['REPORTDESCRIPTION'][j]
    dic_temp['Threat_Score_Person_0'] = sum
    dic_temp['Threat_Score_Person_3'] = sum + (3 * count_names(df_reports['PERSONS'][j]))
    dic_temp['Threat_Score_Person_6'] = sum + (6 * count_names(df_reports['PERSONS'][j]))
    dic_temp['Threat_Score_Person_10'] = sum + (10 * count_names(df_reports['PERSONS'][j]))
    dic_temp['Persons_Num'] = count_names(df_reports['PERSONS'][j])
    dic_temp['Persons'] = df_reports['PERSONS'][j]
    dic_temp['Places'] = df_reports['PLACES'][j]
    dic_temp['Date'] = df_reports['REPORTDATE'][j]
    dic_temp['Keywords'] = keys_words
    dic_temp['Keywords_Score'] = each_word_score

    dic[j] = dic_temp


In [ ]:
answer = final_threat_score.index(max(final_threat_score))
print(answer)

11


In [ ]:
Dataset_report_descriptions["REPORTDESCRIPTION"][answer]

"A tip from an anonymous source directed investigators to a residence at 2111 Troost Ave. in Kansas City, Missouri. This source said that explosives were being manufactured at this residence by some terrorists who intended to blow up several buildings in the Kansas City Area. A warrant was obtained to search this residence on evidence provided by an examination of items found in garbage sacks that were picked up by an FBI officer posing as a garbage collector. Traces of chemicals necessary to produce explosives were found on items in this garbage. When this residence was searched on 22 December, 2003, one man, who gave his name as Walter Smith, was the only person at home. While the investigation was proceeding, another man entered and was apprehended. Subsequent identity checks revealed this man to be Jamal al Quso, who had come from Chicago to Kansas City. What was found at this residence were batches of hexamethylene triperoxide diamine [HMTD] and cyclotrimethylene trinitramine [RDX

In [ ]:
import heapq

heapq.nlargest(4, final_threat_score)

[102, 94, 78, 72]

In [ ]:
answers = [final_threat_score.index(x) for x in heapq.nlargest(5, final_threat_score)]
answers

[11, 52, 71, 33, 73]

In [ ]:
_ = [print(Dataset_report_descriptions["REPORTDESCRIPTION"][x], "\n") for x in answers]

In [ ]:
import json

with open("Dataset.json", "w") as outfile:
    json.dump(dic, outfile)

In [ ]:
print("The End")

The End
